In [4]:
import os
data_home = "/ssd2/arthur/TREC2019/data/"
run_file = "/ssd2/arthur/terrier-core/var/results/run.msmarco_docs.bm25.res"
assert os.path.isfile(run_file)

In [7]:
import os
import gzip
import csv
import random
from collections import defaultdict
split = 'dev'
queries_file = os.path.join(data_home, f"msmarco-doc{split}-queries.tsv.gz")
lookup_file = os.path.join(data_home, "msmarco-docs-lookup.tsv.gz")
qrels_file = os.path.join(data_home, f"msmarco-doc{split}-qrels.tsv")

querystring = {}
with gzip.open(queries_file, 'rt', encoding='utf-8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [topicid, querystring_of_topicid] in tsvreader:
        querystring[topicid] = querystring_of_topicid
        
docoffset = {}
with gzip.open(lookup_file, 'rt', encoding='utf-8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [docid, _, offset] in tsvreader:
        docoffset[docid] = int(offset)
qrel = {}
with open(qrels_file, 'r', encoding='utf8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [topicid, _, docid, rel] in tsvreader:
        assert rel == "1"
        if topicid in qrel:
            qrel[topicid].append(docid)
        else:
            qrel[topicid] = [docid]

def getcontent(docid, f):
    """getcontent(docid, f) will get content for a given docid (a string) from filehandle f.
    The content has four tab-separated strings: docid, url, title, body.
    """

    f.seek(docoffset[docid])
    line = f.readline()
    assert line.startswith(docid + "\t"), \
        f"Looking for {docid}, found {line}"
    return line.rstrip()
docs_file = os.path.join(data_home, "msmarco-docs.tsv")
output_file = os.path.join(data_home, "bert_for_rerank.tsv")

In [10]:
!head $output_file

174249-D3126541	does xpress bet charge to deposit money in your account	http://www.howtodiys.com/elitebet-kenya-paybill-number/ Elitebet Kenya Paybill Number: Start Betting Games Today Bet in KenyaElitebet Kenya Paybill Number: Start Betting Games TodayBy howtodoit - October 11, 2016 113749tweetKlitebet KenyaYou can use Elitebet Kenya Paybill Number to deposit money to your account and start betting games today. This is one of the first online betting sites in Kenya and east Africa. Itâs not as mobile friendly as other popular betting sites in Kenya. Elitebet site is powered by Oxygen8 East Africa Ltd. This is a company incorporated in Kenya and licensed by BCLB, Betting Control and Licensing Board of Kenya. Just like there is Elitebet in Kenya there is Elitebet Uganda. This is one of the betting sites that were launched just after Sportpesa. Here is a guide on How to Play Sportpesa Kenya Jackpot, Double Chance & Multibet.Elitebet Kenya Sign Up and LoginThe process for Elitebet Kenya

In [12]:
!wc -l $output_file

50000 /ssd2/arthur/TREC2019/data/bert_for_rerank.tsv


In [8]:
output_line_format = "{}-{}\t{}\t{}\t{}\n" #query_id, document_id, querystring, document text
with open(run_file, 'r') as inf, open(docs_file, 'r') as f, open(output_file, 'w') as outf:
    for counter, line in enumerate(inf):
        if counter%1000==0:
            print(counter)
        [topic_id, _, doc_id, ranking, score, _] = line.split()
        is_relevant = doc_id in qrel[topic_id]
        
        query = querystring[topic_id]
        document = getcontent(doc_id, f)
        text_to_keep = ' '.join(document.split("\t")[1:])
        outf.write(output_line_format.format(topic_id, doc_id, query, text_to_keep, int(is_relevant)))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [9]:
counter

49999